In [1]:
import IPython
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sympy
import datetime

# для расчетов над tdb
from pycalphad import Database, equilibrium, variables as v, binplot

# для MCMC расчетов
import pymc as pm  # пакет для MCMC расчетов 
import arviz as az # пакет для работы с типом данных arviz
import pytensor
import pytensor.tensor as pt
# import theano
# theano.config.exception_verbosity = 'high' # должно выдавать подробное описание ошибки, но не помогает

import aesara

import seaborn as sns

# пути к tdb
cc10_path = "tdbs/CoCr-01Oik_with_new_functions.tdb"

print(f"Running on PyMC v{pm.__version__}") # 5.1.2
print(f"Running on NumPy v{np.__version__}") # 1.22.1
print(f"Running on ArviZ v{az.__version__}") # 0.12.1

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Running on PyMC v5.1.2
Running on NumPy v1.22.1
Running on ArviZ v0.12.1


# Функции и данные

In [2]:
def from_xarray_to_pandas(xarray_data, phase_str):
    cr_tuple = xarray_data.X.sel(component = 'CR').data[0][0]
    phase_tuple = xarray_data.Phase.data[0][0]
    t_tuple = xarray_data.T.data
    
    # print('- t_tuple', len(t_tuple))
    
    # переписать короче
    df_res = pd.DataFrame()
    df_res['T'] = ''
    df_res['phase'] = ''
    df_res['cr_conc'] = ''
    
    df = pd.DataFrame()
    df['T'] = t_tuple
    df['phase_1'] = ''
    df['phase_2'] = ''
    df['phase_3'] = ''
    df['cr_conc_1'] = ''
    df['cr_conc_2'] = ''
    df['cr_conc_3'] = ''
    
    # можно ли как то иначе это сделать?
    for i in range(len(t_tuple)):
        df.iloc[i, df.columns.get_loc('phase_1')] = phase_tuple[i][0][0]
        df.iloc[i, df.columns.get_loc('phase_2')] = phase_tuple[i][0][1]
        df.iloc[i, df.columns.get_loc('phase_3')] = phase_tuple[i][0][2]
        df.iloc[i, df.columns.get_loc('cr_conc_1')] = cr_tuple[i][0][0]
        df.iloc[i, df.columns.get_loc('cr_conc_2')] = cr_tuple[i][0][1]
        df.iloc[i, df.columns.get_loc('cr_conc_3')] = cr_tuple[i][0][2]
        
    
    # костыль, заменить на автоматическое определение количества фаз и нужного столбца
    
    df_1 = df[(df['phase_1'] == phase_str)]
    df_1 = df_1[['phase_1', 'cr_conc_1', 'T']]
    df_1.rename(columns = {'phase_1':'phase', 'cr_conc_1':'cr_conc'}, inplace = True)
    
    # print('df_1', df_1)
    
    df_2 = df[(df['phase_2'] == phase_str)]
    df_2 = df_2[['phase_2', 'cr_conc_2', 'T']]
    df_2.rename(columns = {'phase_2':'phase', 'cr_conc_2':'cr_conc'}, inplace = True)
    
    # print('df_2', df_2)
    
    df_3 = df[(df['phase_3'] == phase_str)]
    df_3 = df_3[['phase_3', 'cr_conc_3', 'T']]
    df_3.rename(columns = {'phase_3':'phase', 'cr_conc_3':'cr_conc'}, inplace = True)
    
    # print('df_3', df_3)
    
    df_res = pd.concat([df_1, df_2, df_3])
    # df_res = df_2
    
    # print('- df_res rows', df_res.shape[0])
    
    return df_res

In [3]:
db10 = Database(cc10_path)
db10.symbols

{'ZERO': Piecewise((0.0, And(T < 6000.0, 298.15 <= T)), (0, True)),
 'UN_ASS': Piecewise((0.0, And(T < 6000.0, 298.15 <= T)), (0, True)),
 'R': Piecewise((8.31451, And(T < 6000.0, 298.15 <= T)), (0, True)),
 'GHSERCO': Piecewise((310.241 + 133.36601*T - 25.0861*T*log(T) + 72527.0*T**(-1.0) - 0.002654739*T**2.0 - 1.7348e-07*T**3.0, And(T < 1768.0, 298.15 <= T)), (-17197.666 + 253.28374*T - 40.5*T*log(T) + 9.3488e+30*T**(-9.0), And(T < 6000.0, 1768.0 <= T)), (0, True)),
 'GLIQCO': Piecewise((15085.037 + GHSERCO - 8.931932*T - 2.19801e-21*T**7.0, And(T < 1768.0, 298.15 <= T)), (-846.61 + 243.599944*T - 40.5*T*log(T), And(T < 6000.0, 1768.0 <= T)), (0, True)),
 'GFCCCO': Piecewise((427.59 + GHSERCO - 0.615248*T, And(T < 6000.0, 298.15 <= T)), (0, True)),
 'GBCCCO': Piecewise((2938.0 + GHSERCO - 0.7138*T, And(T < 6000.0, 298.15 <= T)), (0, True)),
 'GHSERCR': Piecewise((-8856.94 + 157.48*T - 26.908*T*log(T) + 139250.0*T**(-1.0) + 0.00189435*T**2.0 - 1.47721e-06*T**3.0, And(T < 2180.0, 298.1

In [4]:
df_sigma_fcc = pd.read_excel('emp_data/sigma_fcc_allibert.xls')
# df_sigma_bcc = pd.read_excel('emp_data/sigma_bcc_allibert.xls')
df_sigma_hcp = pd.read_excel('emp_data/sigma_hcp_allibert.xls')

df_sigma_fcc = pd.concat([df_sigma_fcc, df_sigma_hcp])

df_sigma_fcc['T'] = df_sigma_fcc['T'].round(2)
df_sigma_fcc['cr_conc'] = df_sigma_fcc['cr_conc'].round(6)
df_sigma_fcc_sigma_old = df_sigma_fcc[(df_sigma_fcc['phase'] == 'sigma_old')].reset_index()
df_sigma_fcc_sigma_old

,index,cr_conc,T,phase
0,6,0.532019,1321.35,sigma_old
1,7,0.542469,1226.97,sigma_old
2,8,0.547749,1471.91,sigma_old
3,9,0.554050,1370.79,sigma_old
4,10,0.556156,1269.66,sigma_old
5,11,0.559283,1422.47,sigma_old
6,7,0.541762,928.09,sigma_old
7,8,0.545313,1020.22,sigma_old
8,9,0.551960,1069.66,sigma_old
9,10,0.558231,973.03,sigma_old


In [5]:
db10 = Database(cc10_path)

# phases10 = list(db10.phases.keys())
press = 101325
elements = ['CR', 'CO', 'VA']
el_cnt = 1

df_sigma_fcc_sigma_old = df_sigma_fcc_sigma_old.sort_values('T')

T = df_sigma_fcc_sigma_old['T'].to_numpy()
phase = 'SIGMA_OLD'
# phases = ['FCC_A1', 'HCP_A3', 'SIGMA_OLD']

y_obs = df_sigma_fcc_sigma_old['cr_conc'].values
conditions = {v.X('CR'):0.5, v.P: 101325, v.T: T, v.N: el_cnt}
parameters_list = ['SIGMA_OLD_COCRCO_0', 'SIGMA_OLD_COCRCO_1', 'SIGMA_OLD_COCRCR_0', 'SIGMA_OLD_COCRCR_1']

print('T', T)
print('y_obs', y_obs)
# print('phases', phases)
print('phase', phase)

T [ 928.09  973.03 1020.22 1069.66 1121.35 1168.54 1217.98 1226.97 1269.66
 1321.35 1370.79 1422.47 1471.91]
y_obs [0.541762 0.558231 0.545313 0.55196  0.559991 0.565255 0.560713 0.542469
 0.556156 0.532019 0.55405  0.559283 0.547749]
phase SIGMA_OLD


# Class

In [6]:
# define a pytensor Op for our likelihood function
class LogLike(pt.Op):
#     определяем тип входящих и исходящих данных
    itypes = [pt.dvector]  # expects a vector of parameter values when called
    otypes = [pt.fvector]  # outputs a single scalar value (the log likelihood)

    def __init__(self, db, conditions, phase, elements, y_obs):
        self.db_tdb = db
        self.conditions_dict = conditions
        self.phases_list = []
        self.phase_str = phase
        self.elements_list = elements
        self.y_obs_nparray = y_obs
        
        self.y_eqs = []
        self.likelihoods = []

    def perform(self, node, inputs, outputs):
        
        (theta,) = inputs  # this will contain my variables
        
        COCRCO_0, COCRCO_1, COCRCR_0, COCRCR_1 = theta

        self.phases_list = list(self.db_tdb.phases.keys())
        
        new_parameters = {
         'SIGMA_OLD_COCRCO_0' : COCRCO_0
         ,'SIGMA_OLD_COCRCO_1': COCRCO_1
         ,'SIGMA_OLD_COCRCR_0': COCRCR_0
         ,'SIGMA_OLD_COCRCR_1': COCRCR_1
        }
        
        # print('** theta', theta)
        # try: 
        y_eq = from_xarray_to_pandas(equilibrium(self.db_tdb
                                            , self.elements_list
                                            , self.phases_list
                                            , self.conditions_dict
                                            , parameters = new_parameters
                                        ), self.phase_str)['cr_conc'].astype(float).to_numpy() # cr_conc - нужно абстрагировать
            # print('** try y_eq', y_eq.size, y_eq)
        # except:
            # y_eq = 0.0
            # print('** except y_eq', y_eq.size, y_eq)
        # start
        # try:
        #     likelihood = np.array(- np.sum((self.y_obs_nparray - y_eq)**2))
        #     # print('** try y_obs', self.y_obs_nparray)
        #     # print('** try y_eq', y_eq)
        #     # print('** try likelihood', likelihood)
        # except:
        #     likelihood = 0.0
        #     # print('** except likelihood', likelihood)
        self.y_eqs.append(y_eq)
        # self.likelihoods.append(likelihood)
        
        # outputs[0][0] = likelihood
        # end

        outputs[0][0] = y_eq
                             
                             

# Model

In [7]:
pytensor.config.exception_verbosity = 'high' 

test_model = pm.Model()

logl = LogLike(db10, conditions, phase, elements, y_obs)

with test_model:
    # uniform priors on m and c
    COCRCO_0 = pm.Normal("SIGMA_OLD_COCRCO_0", mu=-103863.0, sigma=1)
    COCRCO_1 = pm.Normal("SIGMA_OLD_COCRCO_1", mu=47.47, sigma=1)
    COCRCR_0 = pm.Normal("SIGMA_OLD_COCRCR_0", mu=-248108.8, sigma=1)
    COCRCR_1 = pm.Normal("SIGMA_OLD_COCRCR_1", mu=79.12, sigma=1) 
    


    # theta = [COCRCO_0, COCRCO_1, COCRCR_0, COCRCR_1]
    theta = pt.as_tensor_variable([COCRCO_0, COCRCO_1, COCRCR_0, COCRCR_1])
    # theta_2 = np.array([COCRCO_0, COCRCO_1, COCRCR_0, COCRCR_1])
    
    # likelihood = pm.DensityDist("likelihood", logp = logl_2, observed=y_obs)
    
    # likelihood_2 = pm.Potential("likelihood", logl(theta))
    # pm.HalfNormal("obs", logl(theta), observed=y_obs)#, 
    obs = pm.Deterministic("obs", logl(theta))
    pm.Normal("y_obs", mu=obs, sigma = 0.001, observed=y_obs)
    
    idata = pm.sample_prior_predictive(samples=5)

    
    # trace = pm.sample(draws=2000, tune=500, idata_kwargs={"log_likelihood": True}, progressbar=True)

Sampling: [SIGMA_OLD_COCRCO_0, SIGMA_OLD_COCRCO_1, SIGMA_OLD_COCRCR_0, SIGMA_OLD_COCRCR_1, y_obs]


In [8]:
idata

Inference data with groups:
	> prior
	> prior_predictive
	> observed_data

In [9]:
t = logl.y_eqs
t

[array([0.5358742 , 0.53596563, 0.53629348, 0.53688072, 0.53775472,
        0.53878399, 0.54010408, 0.54037146, 0.54176075, 0.54776084,
        0.55440741, 0.56113146, 0.56739416]),
 array([0.53513373, 0.53516012, 0.53541461, 0.53591851, 0.536697  ,
        0.53763014, 0.53883842, 0.53908407, 0.54036328, 0.54689796,
        0.5534698 , 0.56011157, 0.56629011]),
 array([0.53926057, 0.53948396, 0.53995736, 0.54070659, 0.54176288,
        0.54297292, 0.54450085, 0.54480842, 0.54640039, 0.55120909,
        0.55789778, 0.56467486, 0.57099901]),
 array([0.53248245, 0.53248684, 0.53272126, 0.53320736, 0.5339707 ,
        0.53489312, 0.53609349, 0.53633808, 0.53761394, 0.54444519,
        0.55113084, 0.55789021, 0.56418083]),
 array([0.5363823 , 0.53644632, 0.5367416 , 0.53729012, 0.53811791,
        0.53909908, 0.54036163, 0.54061766, 0.54194871, 0.54813635,
        0.55470448, 0.56134474, 0.56752447])]

In [10]:
len(t)

5

In [11]:
idata.prior_predictive['y_obs'][0][4]

<xarray.DataArray 'y_obs' (y_obs_dim_0: 13)>
array([0.5361104 , 0.53837723, 0.53673433, 0.53879647, 0.53837251,
       0.54035932, 0.54078457, 0.54165724, 0.54312984, 0.546897  ,
       0.55389303, 0.56115088, 0.56656451])
Coordinates:
    chain        int32 0
    draw         int32 4
  * y_obs_dim_0  (y_obs_dim_0) int32 0 1 2 3 4 5 6 7 8 9 10 11 12

In [12]:
idata.observed_data['y_obs']

<xarray.DataArray 'y_obs' (y_obs_dim_0: 13)>
array([0.541762, 0.558231, 0.545313, 0.55196 , 0.559991, 0.565255,
       0.560713, 0.542469, 0.556156, 0.532019, 0.55405 , 0.559283,
       0.547749])
Coordinates:
  * y_obs_dim_0  (y_obs_dim_0) int32 0 1 2 3 4 5 6 7 8 9 10 11 12

In [13]:
# выключила через 30 минут работы
with test_model:
    trace = pm.sample(5, tune=5, idata_kwargs={"log_likelihood": True})

Only 5 samples in chain.
c:\Users\emgebrishvili\AppData\Local\Programs\Python\Python310\lib\site-packages\multipledispatch\dispatcher.py:27: AmbiguityWarning: 
Ambiguities exist in dispatched function _unify

The following signatures may result in ambiguous behavior:
	[ConstrainedVar, object, Mapping], [object, ConstrainedVar, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, Var, Mapping]
	[ConstrainedVar, object, Mapping], [object, ConstrainedVar, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, Var, Mapping]


Consider making the following additions:

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)
  warn(warning_text(dispatcher.name, ambiguities), AmbiguityWarning)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [SIGMA_OLD

Sampling 4 chains for 5 tune and 5 draw iterations (20 + 20 draws total) took 29 seconds.
The number of samples is too small to check convergence reliably.


In [14]:
with test_model:
    idata.extend(trace)

In [15]:
idata

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> prior
	> prior_predictive
	> observed_data

In [16]:
az.summary(idata.prior)

arviz - WARNING - Shape validation failed: input_shape: (1, 5), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
SIGMA_OLD_COCRCR_1,79.123,0.888,78.309,80.573,0.572,0.467,2.0,2.0,NaN
SIGMA_OLD_COCRCR_0,-248108.571,0.940,-248109.579,-248107.237,0.605,0.494,2.0,2.0,NaN
SIGMA_OLD_COCRCO_0,-103862.460,1.637,-103863.936,-103860.479,1.055,0.862,2.0,2.0,NaN
SIGMA_OLD_COCRCO_1,47.493,0.979,45.894,48.423,0.631,0.515,2.0,2.0,NaN
obs[0],0.536,0.002,0.532,0.539,0.002,0.001,2.0,2.0,NaN
obs[1],0.536,0.003,0.532,0.539,0.002,0.001,2.0,2.0,NaN
obs[2],0.536,0.003,0.533,0.540,0.002,0.001,2.0,2.0,NaN
obs[3],0.537,0.003,0.533,0.541,0.002,0.001,2.0,2.0,NaN
obs[4],0.538,0.003,0.534,0.542,0.002,0.001,2.0,2.0,NaN
obs[5],0.539,0.003,0.535,0.543,0.002,0.002,2.0,2.0,NaN


In [17]:
az.summary(idata.posterior)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
SIGMA_OLD_COCRCO_0,-103863.023,0.940,-103864.358,-103861.499,0.214,0.154,19.0,19.0,1.24
SIGMA_OLD_COCRCO_1,52.125,1.667,50.020,54.116,0.380,0.273,19.0,19.0,3.02
SIGMA_OLD_COCRCR_0,-248108.604,0.894,-248109.721,-248106.906,0.204,0.146,19.0,19.0,1.36
SIGMA_OLD_COCRCR_1,78.970,1.925,76.453,81.298,0.438,0.315,19.0,19.0,2.59
obs[0],0.544,0.001,0.543,0.545,0.000,0.000,19.0,19.0,2.53
obs[1],0.544,0.001,0.543,0.545,0.000,0.000,19.0,19.0,2.47
obs[2],0.545,0.001,0.544,0.546,0.000,0.000,19.0,19.0,2.16
obs[3],0.546,0.001,0.545,0.546,0.000,0.000,19.0,19.0,1.97
obs[4],0.547,0.000,0.546,0.547,0.000,0.000,19.0,19.0,1.78
obs[5],0.548,0.000,0.547,0.548,0.000,0.000,19.0,19.0,1.56


In [18]:
with test_model:
    ppc = pm.sample_posterior_predictive(trace)

ValueError: different number of dimensions on data and dims: 3 vs 2